# Knowledge Based Recommender

Knowledge Based recommender are used for items that are rarely purchased.Simply put it is not possible to recommend items based on past purchase.Ex:-Real estate is a once in a lifetime investment.It is not possible to have history of real estate purhcases of exsisting user Nor it is possible to ask them about there real estate history.

In such cases we build something called a Knowledge based recommender system,that asks for specifics of your purchase and it filters though the data base and recommend you with an item.
    
    
<b>Disadvantage<b>

    Suffers from novelty.Sometimes users know what to expect from the result but are taken by surprise by the recommendation
    
    
 


In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
df=pd.read_csv('movies_metadata.csv')
df.head(3)

C:\Users\Vishnu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


# Building a Knowledge based Recommender

Let us build a knowledge based recommender on the following features :
1) Ask the users for genres of movies and he/she is looking for 

2) Ask the duration of the movies

3) Ask the user for timeline of the movie (years)

<b>Using the above features we will recommend a movies based on weighted rating score 

In [3]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
df=df[['title','genres','release_date','runtime','vote_average','vote_count']]
df

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0
...,...,...,...,...,...,...
45461,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,90.0,4.0,1.0
45462,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",2011-11-17,360.0,9.0,3.0
45463,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",2003-08-01,90.0,3.8,6.0
45464,Satan Triumphant,[],1917-10-21,87.0,0.0,0.0


In [5]:
df['release_date']=pd.to_datetime(df['release_date'],errors='coerce')

In [6]:
#extracting year from release date 
df['year']=df['release_date'].apply(lambda x:str(x).split('-')[0] if x != np.nan else np.nan )

In [7]:
#our year feature is object datatype and has null values,let us convert them into integer and remove null
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

In [8]:
df['year']=df['year'].apply(convert_int)

In [9]:
#removing release date feature 
df=df.drop('release_date',axis=1)

In [10]:
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [11]:
# we can see that the values in genres feature are stringified dictionary,
#in order to use them we have to convert them into python dictionary

#let us import literal eval function from ast 
#le us look at an example for it;s use 

from ast import literal_eval
a='[1,2,3]'
print(type(a))

#apllying literal eval
b=literal_eval(a)
print(type(b))

<class 'str'>
<class 'list'>


In [12]:
#befor applying literal eval let us convert all nan values 

df['genres']=df['genres'].fillna('[]')

#apply literal eval to convert into pythin dictionary 

df['genres']=df['genres'].apply(literal_eval)

#convert list of dictionary into list of string 

df['genres']=df['genres'].apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])

In [13]:
df.iloc[0]['genres']

['Animation', 'Comedy', 'Family']

In [15]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995


we can see that genres is now a list of genre names,now what we need to do is explode the genres since it contains more than one genre for a particular movies,We will create muliplte copies of the movis for each genre

In [17]:
df.shape

(45466, 6)

In [18]:
#create a new feature by exploding genres

s=df.apply(lambda x :
          pd.Series(x['genres']),axis=1).stack().reset_index(level=1,drop=True)

<ipython-input-18-02421da765d6>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(x['genres']),axis=1).stack().reset_index(level=1,drop=True)


In [19]:
s

0        Animation
0           Comedy
0           Family
1        Adventure
1          Fantasy
           ...    
45461       Family
45462        Drama
45463       Action
45463        Drama
45463     Thriller
Length: 91106, dtype: object

In [20]:
s.shape

(91106,)

In [21]:
s.name='genre'

In [22]:
#creating a new dataframe by dropping the old genre 

gen_df=df.drop('genres',axis=1).join(s)

In [23]:
gen_df.head()

,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,Animation
0,Toy Story,81.0,7.7,5415.0,1995,Comedy
0,Toy Story,81.0,7.7,5415.0,1995,Family
1,Jumanji,104.0,6.9,2413.0,1995,Adventure
1,Jumanji,104.0,6.9,2413.0,1995,Fantasy


In [24]:
gen_df.shape

(93548, 6)

Let us now a define a function which take the user inputs and the machine filters through our data base and spits out the recommended movie


In [32]:
def build_chart(gen_df, percentile=0.8):
    #Ask for preferred genres
    print("Input preferred genre")
    genre = input()
    #Ask for lower limit of duration
    print("Input shortest duration")
    low_time = int(input())
    #Ask for upper limit of duration
    print("Input longest duration")
    high_time = int(input())
    #Ask for lower limit of timeline
    print("Input earliest year")
    low_year = int(input())
    #Ask for upper limit of timeline
    print("Input latest year")
    high_year = int(input())
    #Define a new movies variable to store the preferred movies. Copy thecontents of gen_df to movies
    movies = gen_df.copy()
    #Filter based on the condition
    movies = movies[(movies['genre'] == genre) &
                    (movies['runtime'] >= low_time) &
                    (movies['runtime'] <= high_time) &
                    (movies['year'] >= low_year) &
                    (movies['year'] <= high_year)]
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    #Only consider movies that have higher than m votes. Save this in a newdataframe q_movies
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x:
                                       (x['vote_count']/(x['vote_count']+m) * x['vote_average'])
                                       + (m/(m+x['vote_count']) * C),axis=1)
    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    return q_movies


In [36]:
build_chart(gen_df)

Input preferred genre
Comedy
Input shortest duration
45
Input longest duration
300
Input earliest year
2000
Input latest year
20018


,title,runtime,vote_average,vote_count,year,genre,score
18465,The Intouchables,112.0,8.2,5410.0,2011,Comedy,8.144003
22841,The Grand Budapest Hotel,99.0,8.0,4644.0,2014,Comedy,7.940208
22131,The Wolf of Wall Street,180.0,7.9,6768.0,2013,Comedy,7.860434
30315,Inside Out,94.0,7.9,6737.0,2015,Comedy,7.860255
40882,La La Land,128.0,7.9,4745.0,2016,Comedy,7.843995
...,...,...,...,...,...,...,...
18101,Jack and Jill,91.0,4.0,619.0,2011,Comedy,4.283863
12358,Meet the Spartans,84.0,3.8,370.0,2008,Comedy,4.277146
9710,Son of the Mask,94.0,3.6,346.0,2005,Comedy,4.154277
12911,Disaster Movie,87.0,3.1,250.0,2008,Comedy,3.962326


In [38]:
#saving the cleaned dataset 
df.to_csv('metadata_clean.csv',index=False)